### Mod 1 Project --- Data Movie

In [1]:
#top 50 lucrative movies from  tn_movie_budgets and bom_movie_gross   (find their ratings)
# most (top50) popular movies from tmdb_movies
# highest(top 50) ratings from imdb_title_ratings

# select common movies from above list
#find their genre, directors

### Import libraries & database

In [2]:
!pip install mysql-connector-python

In [3]:
import pandas as pd
import numpy as np
import mysql.connector
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
con = mysql.connector.connect(user='hbs-student', password='super-secure-password?',
                              host='hbs-ds-060120.c1doesqrid0e.us-east-1.rds.amazonaws.com',
                              database='movies')
cur = con.cursor()

# Data Cleaning

### IMDB

In [5]:
query = ('''
SELECT *
FROM imdb_title_akas imdb_a
JOIN imdb_title_basics imdb_b ON imdb_a.title_id = imdb_b.tconst
JOIN imdb_title_crew imdb_c ON imdb_b.tconst = imdb_c.tconst
JOIN imdb_title_principals imdb_d ON imdb_b.tconst = imdb_d.tconst
JOIN imdb_title_ratings imdb_e ON imdb_b.tconst = imdb_e.tconst
''')
imdb_titles = pd.read_sql_query(query, con)
#joining all IMDB title tables using title ids and name ids

In [6]:
imdb_titles.isna().sum()
#checking for null values

title_id                  0
ordering                  0
title                     0
region                    0
language                  0
types                     0
attributes                0
is_original_title         0
tconst                    0
primary_title             0
original_title            0
start_year                0
runtime_minutes       95038
genres                    0
tconst                    0
directors                 0
writers              139830
tconst                    0
ordering                  0
nconst                    0
category                  0
job                       0
characters                0
tconst                    0
averagerating             0
numvotes                  0
dtype: int64

In [7]:
imdb_titles = imdb_titles.dropna(subset=['runtime_minutes'])
imdb_titles = imdb_titles.dropna(subset=['writers'])
#Removing rows with null values for runtime minutes and writers

In [8]:
query = ('''
SELECT *
FROM imdb_name_basics
''')
imdb_crew = pd.read_sql_query(query, con)

In [9]:
imdb_crew.isna().sum()

nconst                     0
primary_name               0
birth_year            523912
death_year            599865
primary_profession         0
known_for_titles           0
dtype: int64

In [10]:
imdb_crew = imdb_crew.drop(['birth_year','death_year'], axis=1)
#dropping these two columns due to large amount of null values and lack of relevancy for analysis

In [14]:
imdb_crew['known_for_titles']

0         tt0038355,tt0117057,tt0071877,tt0037382
1         tt0049189,tt0057345,tt0054452,tt0059956
2         tt0069467,tt0050986,tt0083922,tt0050976
3         tt0071877,tt0036855,tt0038109,tt0038787
4         tt0034583,tt0033870,tt0037382,tt0043265
                           ...                   
606643                                  tt7722258
606644                                 tt10299418
606645                                  tt4844148
606646                                  tt8739208
606647                                  tt8295580
Name: known_for_titles, Length: 606648, dtype: object

In [15]:
query = ('''
SELECT tconst, primary_title
FROM imdb_title_basics
''')
imdb_key = pd.read_sql_query(query, con)
#Creating a key to identify which title_ids correspond to which titles

In [25]:
imdb_titles.head()

,title_id,ordering,title,region,language,types,attributes,is_original_title,tconst,primary_title,...,writers,tconst,ordering,nconst,category,job,characters,tconst,averagerating,numvotes
0,tt0063540,1,Sangharsh,IN,hi,,alternative transliteration,0,tt0063540,Sunghursh,...,"nm0023551,nm1194313,nm0347899,nm1391276",tt0063540,10,nm0006210,composer,,,tt0063540,7.0,77
1,tt0063540,1,Sangharsh,IN,hi,,alternative transliteration,0,tt0063540,Sunghursh,...,"nm0023551,nm1194313,nm0347899,nm1391276",tt0063540,1,nm0474801,actor,,"[""Kundan S. Prasad"",""Bajrangi""]",tt0063540,7.0,77
2,tt0063540,1,Sangharsh,IN,hi,,alternative transliteration,0,tt0063540,Sunghursh,...,"nm0023551,nm1194313,nm0347899,nm1391276",tt0063540,2,nm0904537,actress,,"[""Munni"",""Laila-E-Aasmaan""]",tt0063540,7.0,77
3,tt0063540,1,Sangharsh,IN,hi,,alternative transliteration,0,tt0063540,Sunghursh,...,"nm0023551,nm1194313,nm0347899,nm1391276",tt0063540,3,nm0756379,actor,,"[""Ganeshi N. Prasad""]",tt0063540,7.0,77
4,tt0063540,1,Sangharsh,IN,hi,,alternative transliteration,0,tt0063540,Sunghursh,...,"nm0023551,nm1194313,nm0347899,nm1391276",tt0063540,4,nm0474876,actor,,"[""Dwarka N. Prasad""]",tt0063540,7.0,77


In [16]:
imdb_key

,tconst,primary_title
0,tt1699720,!Women Art Revolution
1,tt2346170,#1 Serial Killer
2,tt3120962,#5
3,tt8219776,#5
4,tt4470336,#50Fathers
...,...,...
146139,tt9741450,¿Quién Soy?
146140,tt3867932,¿Whisky?
146141,tt7207944,¿Y los tamales?
146142,tt2533534,Ørkenens sønner - Varm luft i canal grande


In [17]:
imdb_crew = imdb_crew.assign(known_for_titles=imdb_crew['known_for_titles'].str.split(',')).explode('known_for_titles')
# Separating the title IDs in the "known for" column

In [24]:
type(imdb_key)

pandas.core.frame.DataFrame

In [28]:
imdb_crew_with_key = imdb_crew.merge(imdb_key, how='left', left_on='known_for_titles', right_on='tconst')
# Trying to match "known for" title codes with film names

In [37]:
imdb_crew_with_key = imdb_crew_with_key.dropna(subset=['tconst','primary_title'])

In [38]:
imdb_crew_with_key.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 791006 entries, 152 to 1668462
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   nconst              791006 non-null  object
 1   primary_name        791006 non-null  object
 2   primary_profession  791006 non-null  object
 3   known_for_titles    791006 non-null  object
 4   tconst              791006 non-null  object
 5   primary_title       791006 non-null  object
dtypes: object(6)
memory usage: 42.2+ MB


### BOM

In [18]:
query = ('''
SELECT * FROM bom_movie_gross
''')
bom = pd.read_sql_query(query, con)

In [19]:
#drop Null value in domestic_gross
bom = bom.dropna(subset =['domestic_gross'])

#convert domestic_gross from float to int
bom.domestic_gross = bom.domestic_gross.map(lambda x: int(x))    

### TN

In [20]:
query_tn = ('''
SELECT * FROM tn_movie_budgets
''')
tn = pd.read_sql_query(query_tn, con)

In [21]:
# convert release_date to date format
tn.release_date = tn.release_date.map(lambda x: dt.datetime.strptime(x, '%b %d, %Y'))

# convert production_budget, domestic_gross, worldwide_gross to date format
tn.production_budget = tn.production_budget.map(lambda x: int(x.replace(',','').replace('$','')))
tn.domestic_gross = tn.domestic_gross.map(lambda x: int(x.replace(',','').replace('$','')))
tn.worldwide_gross = tn.worldwide_gross.map(lambda x: int(x.replace(',','').replace('$','')))

#add profit column
tn['profit'] = tn.worldwide_gross - tn.production_budget

### TMDB

In [22]:
query_tmdb = ('''
SELECT * FROM tmdb_movies
''')
tmdb = pd.read_sql_query(query_tmdb, con)

### RT

In [23]:
query_rt_info = ('''
SELECT * FROM rt_movie_info
''')
rt_info = pd.read_sql_query(query_rt_info, con)

In [24]:
query_rt_reviews = ('''
SELECT * FROM rt_reviews
''')
rt_reviews = pd.read_sql_query(query_rt_reviews, con)

# Bringing Them All Together

# Top 200 Lucrative Movies

In [25]:
# get top 200 lucrative domestic movies
bom_top200_domestic = bom.sort_values('domestic_gross', ascending = False).head(200)   